# Описание данных

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**ЗАДАНИЕ**

* Посчитайте, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитайте количество обзоров и среднюю оценку;
* Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
* Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
* Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
            'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'], 
                                                         db_config['db']) 

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
def select(sql):
    return pd.io.sql.read_sql(sql, con = engine)

In [3]:
books = '''
            SELECT * 
            FROM books
            LIMIT 5
        '''
select(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица `books`(cодержит данные о книгах):
* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.

In [4]:
authors = '''
            SELECT * 
            FROM authors
            LIMIT 5
          '''
select(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица `authors`(cодержит данные об авторах):
* `author_id` — идентификатор автора;
* `author` — имя автора.

In [5]:
publishers = '''
            SELECT * 
            FROM publishers
            LIMIT 5
         '''
select(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица `publishers` (cодержит данные об издательствах):
* publisher_id — идентификатор издательства;
* publisher — название издательства.

In [6]:
ratings = '''
            SELECT * 
            FROM ratings
            LIMIT 5
          '''
select(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица `ratings`(cодержит данные о пользовательских оценках книг):
* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

In [7]:
reviews = '''
            SELECT * 
            FROM reviews
            LIMIT 5
          '''
select(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица `reviews` (cодержит данные о пользовательских обзорах):
* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя автора обзора;
* `text` — текст обзора.

Найдем, сколько книг вышло после 1 января 2000 года.

In [8]:
number_books = '''
            SELECT COUNT(book_id) AS books_cnt 
            FROM books 
            WHERE publication_date > '2000-01-01'
'''
select(number_books)

,books_cnt
0,819


Было выпущено начиная с 1 января 2000 года 819 книг.

Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [9]:
review_valuation=  '''
            SELECT
                books.book_id AS book_id,
                books.title AS title,
                COUNT(DISTINCT review_id) as cnt_reviews,
                AVG(rating) as avg_rating
            FROM
                books 
                FULL JOIN reviews 
                    ON books.book_id = reviews.book_id
                FULL JOIN ratings 
                    ON books.book_id = ratings.book_id
            GROUP BY
                books.book_id
            ORDER BY 
                cnt_reviews DESC
            LIMIT 15

         '''
select(review_valuation)

,book_id,title,cnt_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,854,The Road,6,3.772727
2,207,Eat Pray Love,6,3.395833
3,656,The Book Thief,6,4.264151
4,733,The Giver (The Giver #1),6,3.750000
5,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
7,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
8,497,Outlander (Outlander #1),6,4.125000
9,627,The Alchemist,6,3.789474


Больше всего обзоров у книги Twilight (Twilight #1) со средним рейтингом 3.66.

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [10]:
publishing_house = '''
            SELECT
                publishers.publisher AS publisher_house,
                COUNT(book_id) as cnt_books
            FROM
                publishers 
                FULL JOIN books 
                    ON publishers.publisher_id = books.publisher_id
            WHERE
                books.num_pages > 50
            GROUP BY
                publishers.publisher
            ORDER BY 
                cnt_books DESC
            LIMIT 15 

         '''
select(publishing_house)

,publisher_house,cnt_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,Delta,13


Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц.

Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками.

In [11]:
valuation = '''
              SELECT
                  authors.author AS author,
                  AVG(SUBQ.avg_rating) as avg_author_rating
              FROM
                  authors
                  INNER JOIN (
                        SELECT
                            books.book_id AS book_id,
                            books.author_id AS author_id,
                            AVG(rating) AS avg_rating,
                            COUNT(rating) AS cnt_rating
                        FROM   
                            books 
                            INNER JOIN ratings
                            ON books.book_id = ratings.book_id
                        GROUP BY
                            books.book_id
                        ORDER BY 
                            cnt_rating DESC) AS SUBQ ON authors.author_id = SUBQ.author_id

              WHERE
                  SUBQ.cnt_rating >= 50
              GROUP BY
                  authors.author
              ORDER BY 
                  avg_author_rating DESC
              LIMIT 15
                
         '''
select(valuation)

,author,avg_author_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


J.K. Rowling/Mary GrandPré	 имеет самую  среднюю высокую оценку.

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
avg_reviews = '''
            SELECT ROUND(avg(count)) 
            FROM
            (SELECT
                COUNT(text)
            FROM 
                reviews
            FULL OUTER JOIN (SELECT username,
                            COUNT(rating_id) as cnt
            FROM 
                ratings
            GROUP BY
                    username) AS cnt_rating
            ON cnt_rating.username = reviews.username
            where cnt > 50
            GROUP BY reviews.username) as count;                                  
        '''
select(avg_reviews)

,round
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок равно 24.

# Общий вывод

* Начиная с 1 января 2000 года было выпущено 819 книг;
* Больше всего обзоров у книги Twilight (Twilight #1) со средним рейтингом 3.66;
* Издательство Penguin Books выпустило наибольшее число книг (42) толще 50 страниц;
* Среднее количество обзоров от пользователей, которые поставили больше 50 оценок равно 24.